In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
import matplotlib.pyplot as plt

In [4]:
def train_classifier(model, lines, output, batches, iterations=1):
    errors = []
    lossfunc = nn.BCELoss()
    optimizer = optim.Adagrad(model.parameters())
    for _ in range(iterations):
        for batch in batches:
            try:
                if len(batch) == 20:
                    input = torch.cuda.LongTensor([lines[n] for n in batch])
                    truth = torch.cuda.FloatTensor([output[n] for n in batch])
                    truth[truth == 2] = 0
                    pred = model(input)
                    model.zero_grad()
                    loss = lossfunc(pred, truth)
                    loss.backward()
                    optimizer.step()
                    errors.append(loss.data)
            except:
                print('ad')
            print(batches.index(batch), end='\r')
            plt.plot(errors)

In [ ]:
def get_rouge_r1(summary_folder, document_folder, docid, pred):
    s = {}
    with open(summary_folder + str(docid)) as f:
        summary = f.readlines()
        for line in summary:
            words = line.split()
            for word in words:
                if word in s: s[word] += 1
                else: s[word] = 1

    pred_summary_words = []
    with open(document_folder + str(docid)) as f:
        lines = f.readlines()
        for line, pred_ in zip(lines, pred):
            if pred_ == 1:
                pred_summary_words.extend(line.split())
    
    

In [5]:
def get_summaries(model, lines_, batches, doc_folder, write_folder):
    for batch in batches:
        input = torch.cuda.LongTensor([lines_[n] for n in batch])
        pred = model(input)
        _idx = torch.sort(pred, descending=True)[1].data
        for lines, docid in zip(_idx, batch):
            with open(doc_folder + str(docid)) as f:
                content = f.readlines()
                selected_lines = [content[l] for l in lines[0:3]]
                with open(write_folder + str(docid), 'w+') as f2:
                    f2.write(str(lines[0:3]) + '\n')
                    [f2.write(line) for line in selected_lines]

In [ ]:
def get_file(doc_id, write_folder):
    print(open(write_folder + str(doc_id)).read())

In [ ]:
def computeRouge(folder1, folder2):
    # rouge_args="-e /home/ramkishore.s/ROUGE-1.5.5/data/ -a -c 95 -m -n 2 -w 1.2"
    from pyrouge import Rouge155
    r = Rouge155()
    r.system_dir = folder1
    r.model_dir = folder2
    r.system_filename_pattern = '(\d+)'
    r.model_filename_pattern = '#ID#'

    output = r.convert_and_evaluate()
    output = r.output_to_dict(output)
    return output['rouge_1_f_score_ce'], output['rouge_2_f_score_ce'], output['rouge_l_f_score_ce']